In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%pip install statsmodels --upgrade

You should consider upgrading via the '/Users/enrico/Documents/Faculdade/TCC Local/Projeto/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from methods.ar import ar, arima
from methods.naive import average_forecast, all_true, all_false
from methods.lstm import lstm
from methods.mlp import mlp
from methods.mlp import mlp_keras
from methods.cnn import cnn
from methods.classifiers import knn, gaussian, random_forest
from methods.runner import Runner
import functools


In [ ]:
# ar -> n_steps = 2, 4, 8
# arima -> n_steps = 2, 4, 8; order = 2, 4, 8; differencing = 1, 2, 3
# KNN -> n_steps = 2, 4, 8; n_neighbours = 2, 4, 8; distances = 1, 2, 3, 4, 5; 
# MLP -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200
# LSTM -> n_steps = 2, 4, 8; epochs = 50, 100, 200; layers = 1, 2, 3; cells = 50, 100, 200

models = {}
epochs = [50, 100, 200]
layers = [1, 2, 3]
cells = [50, 100, 200]
orders = [2, 4, 8]
differencing = [1, 2, 3]
n_neighbours = [2, 4, 8]
distances = [1, 2, 3, 4, 5]

for order in orders:
	for diff in differencing:
		models[f"ARIMA_{order}_{diff}"] = functools.partial(arima, order=order, differencing=diff)

for n in n_neighbours:
	for d in distances:
		models[f"KNN_{n}_{d}"] = functools.partial(knn, n_neighbors=n, distance=d)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"MLP_{epoch}_{layer}_{cell}"] = functools.partial(mlp_keras, epochs=epoch, layers=layer, cells=cell)

for epoch in epochs:
	for layer in layers:
		for cell in cells:
			models[f"LSTM_{epoch}_{layer}_{cell}"] = functools.partial(lstm, epochs=epoch, layers=layer, cells=cell)

In [5]:
# models = {
# 	"Naive": average_forecast,
# 	"All True": all_true,
# 	"All False": all_false,
# 	# "AR": ar,
# 	"ARIMA": arima,
# 	# "MLP": mlp,
# 	# "KNN": knn,
# 	# "Gaussian": gaussian,
# 	# "RandomForest": random_forest,
# 	# "MLP Keras": mlp_keras,
# 	# "CNN": cnn,
# 	# "LSTM": lstm,
# 	# "Stacked LSTM": stacked_lstm
# }

In [11]:
models = {}
# models["MLP_200_3_200"] = functools.partial(mlp_keras, epochs=200, layers=3, cells=200)
models["LSTM_100_1_100"] = functools.partial(lstm, cells=50, layers=1, epochs=100, verbose=1, n_classes=3)
# models["LSTM_100_1_100"] = functools.partial(lstm, cells=50, layers=1, epochs=100, verbose=1)
# models["All True"] = all_true
# models["All False"] = all_false

In [13]:
def callback(self):
	print(self)

runner = Runner(models, callback = callback)
# runner.run_all("accuracy", delta_separator = 0.056)
runner.run_all("accuracy")
# runner.run("accuracy")


# runner.run_transfer("accuracy")
# runner.continue_run_transfer("202205022349", "accuracy")
# runner.continue_run_transfer("202205011401", "accuracy")

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 50)                10400     
                                                                 
 dense_7 (Dense)             (None, 3)                 153       
                                                                 
Total params: 10,553
Trainable params: 10,553
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
61/61 [==============================] - 5s 6ms/step - loss: 0.8576 - accuracy: 0.7668
Epoch 2/100
61/61 [==============================] - 0s 7ms/step - loss: 0.5743 - accuracy: 0.7953
Epoch 3/100
61/61 [==============================] - 0s 6ms/step - loss: 0.5669 - accuracy: 0.7953
Epoch 4/100
61/61 [==============================] - 0s 6ms/step - loss: 0.5655 - accuracy: 0.7953
Epoch 5/100
61/61 [==============================]

In [14]:
import pandas as pd
results = pd.read_csv('results/202205151256.csv')

results.head()

,y,y_pred,y_label,time,file,model
0,0,0,202111,3.142892,All.csv,LSTM_100_1_100
1,0,0,202112,3.142892,All.csv,LSTM_100_1_100
2,2,2,202201,3.142892,All.csv,LSTM_100_1_100
3,1,0,202202,3.142892,All.csv,LSTM_100_1_100
4,0,0,202006,3.142892,All.csv,LSTM_100_1_100


In [19]:
from methods.preprocess import Preprocess

df = results.copy()
preprocess = Preprocess()
models = list(df.model.unique())
accuracies = {}

for model in models:
	df_model = df[df["model"] == model]
	y = df_model["y"].tolist()
	y_pred = df_model["y_pred"].tolist()

	metrics = preprocess.evaluate(y, y_pred)
	accuracies[model] = metrics["fmeasure"]


print(sorted(accuracies.items(), key=lambda x: x[1], reverse=True))

[('LSTM_100_1_100', 0.6965591397849463)]
